In [18]:
import requests
from bs4 import BeautifulSoup
import os
import re

# URLs
base_url = 'https://kcserevision.com/'
target_url = 'https://kcserevision.com/grade-6-kicd-curriculum-designs-pdf/'

# Your login credentials
credentials = {
    'swpm_user_name': 'njoroge12',
    'swpm_password': 'njoroge'
}

# Create a session object
session = requests.Session()

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

# Attempt to login
response = session.post(base_url, headers=headers, data=credentials)



In [19]:
response

<Response [200]>

In [19]:
import requests
from bs4 import BeautifulSoup
import os
import re

def extract_google_drive_links(session, url, headers):
    """Extracts Google Drive links, their titles, and the source URL from a given page URL."""
    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    drive_links = [(a.get_text(strip=True), a['href'], url.replace('https://kcserevision.com', '')) for a in soup.find_all('a', href=re.compile(r"https://drive\.google\.com/.*"))]
    return drive_links

# URLs
base_url = 'https://kcserevision.com/'
initial_url = 'https://kcserevision.com'

# Create a session object
session = requests.Session()

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

# Directory to save the file
processed_dir = '../data/processed/'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Filename for the output
output_filename = os.path.join(processed_dir, 'google_drive_links.txt')

# Set to keep track of unique URLs
unique_urls = set()

# Process and save unique links
with open(output_filename, 'w') as file:
    # Get the initial page with the grade links
    response = session.get(initial_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all links that start with "GRADE "
    grade_links = [a['href'] for a in soup.find_all('a', href=True) if a.text.startswith("GRADE ")]

    for grade_url in grade_links:
        drive_links = extract_google_drive_links(session, grade_url, headers)
        for title, link, source_url in drive_links:
            full_url = f"{link}"
            if full_url not in unique_urls:
                unique_urls.add(full_url)
                file.write(f"{link}\n")
        print(f"Processed {grade_url}")
        
print(f"Google Drive links from all grade pages saved to {output_filename}")


Processed https://kcserevision.com/grade-1-2-3-4-cbc-lesson-plans/
Processed https://kcserevision.com/grade-1-2-3-4-5-6-7-8-cbc-revision/
Processed https://kcserevision.com/2024-grade-7-8-cbc-junior-secondary-resources/
Processed https://kcserevision.com/grade-8-junior-secondary-schemes-of-work/
Processed https://kcserevision.com/grade-8-junior-secondary-exams/
Processed https://kcserevision.com/grade-8-junior-secondary-assignments/
Processed https://kcserevision.com/grade-8-junior-secondary-notes/
Processed https://kcserevision.com/grade-8-junior-secondary-lesson-plans/
Processed https://kcserevision.com/grade-8-curriculum-designs-kicd/
Processed https://kcserevision.com/2024-grade-8-assessment-and-scoresheet/
Processed https://kcserevision.com/grade-8-term-1-2-3-opener-mid-end-exams/
Processed https://kcserevision.com/2024-grade-7-schemes-of-work-term-1-2-3/
Processed https://kcserevision.com/2024-jss-grade-7-term-1-2-3-exams/
Processed https://kcserevision.com/2024-grade-7-jss-notes

In [25]:
import re

# Re-running the code due to execution state reset
# Reading the Google Drive links file to process for removing duplicates based on file or folder ID
file_path = '../data/processed/google_drive_links_v1.txt'

# Function to extract unique Google Drive file or folder IDs from the URL
def extract_unique_ids(file_path):
    unique_ids = set()
    unique_lines = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Extracting file/folder ID from the Google Drive URL
            match = re.search(r'drive\.google\.com/(?:file/d/|folders/)([a-zA-Z0-9_-]+)', line)
            if match:
                file_id = match.group(1)
                # Add line to unique list if ID is unique
                if file_id not in unique_ids:
                    unique_ids.add(file_id)
                    unique_lines.append(line)

    return unique_lines

# Extracting unique lines based on Google Drive file or folder IDs
unique_google_drive_links = extract_unique_ids(file_path)

# Count of unique links
len(unique_google_drive_links)

## Write unique links to file
# Directory to save the file
processed_dir = '../data/processed/'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Filename for the output
output_filename = os.path.join(processed_dir, 'google_drive_links_v1.unq.txt')

# Write unique links to file
with open(output_filename, 'w') as file:
    for line in unique_google_drive_links:
        file.write(line)


In [26]:
import re
from collections import defaultdict
import os
#Regular expressions for extracting subject and type
subject_pattern = re.compile(r"GRADE \d+ TERM \d+ (\w+[\w\s]*)(?: SCHEMES| LESSON PLANS| NOTES| EXAMS| ACTIVITIES| ASSIGNMENTS)?", re.IGNORECASE)
type_pattern = re.compile(r"(SCHEMES|LESSON PLANS|NOTES|EXAMS|ACTIVITIES|ASSIGNMENTS)", re.IGNORECASE)

# Organize links by subject and type
organized_links = defaultdict(lambda: defaultdict(list))
for line in unique_google_drive_links:
    subject_match = subject_pattern.search(line)
    type_match = type_pattern.search(line)

    if subject_match and type_match:
        subject = subject_match.group(1).strip().upper()
        type_ = type_match.group(1).strip().upper()
        organized_links[subject][type_].append(line)

# Writing to files
output_dir = "data/processed/organized_links"
os.makedirs(output_dir, exist_ok=True)

for subject, types in organized_links.items():
    subject_dir = os.path.join(output_dir, subject)
    os.makedirs(subject_dir, exist_ok=True)

    for type_, links in types.items():
        file_name = os.path.join(subject_dir, f"{type_}.txt")
        with open(file_name, 'w') as file:
            for link in links:
                file.write(link + '\n')

# Return a sample of the organized structure for demonstration
sample_output = {subject: dict(types) for subject, types in list(organized_links.items())[:2]}
sample_output


{'AGRICULTURE SCHEMES': {'SCHEMES': ['GRADE 6 TERM 1 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/11Csc06oFzmsRvNZ7vFOuRv2GPfmzQZ7g/view?usp=drive_web\n',
   'GRADE 6 TERM 2 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/1EmDBkEU8yoQ-0fSTQkbvNP3qPu4zK5AH/view?usp=drive_web\n',
   'GRADE 6 TERM 3 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/1PiXakw8z3k2bmtsMzBIcBrKVv4qDGRv9/view?usp=drive_web\n',
   'GRADE 5 TERM 1 AGRICULTURE SCHEMES (1).docx: https://drive.google.com/file/d/1D2OYub-FPNivGVaBRrt-nXc_eNa7P3Tf/view?usp=drive_web\n',
   'GRADE 5 TERM 1 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/1PfF9tDxXwvCMBNDwsLnpgSA5isjuepfX/view?usp=drive_web\n',
   'GRADE 5 TERM 2 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/1eV9bnpbJFwXt0GF47Te9RDw_vix_s71R/view?usp=drive_web\n',
   'GRADE 5 TERM 3 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/1ln18_76pFjLbPBnp7NM2R8qKAOvI061m/view?usp=drive_web\n',
   'GRADE 4 TERM 1 AGRIC

In [3]:
# Check if login was successful by looking at cookies or response content
if 'swpm_session' in session.cookies:
    print("Logged in successfully.")

    # Get the page with the files
    response = session.get(target_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all Google Drive links and their titles
    drive_links = [(a.get_text(strip=True), a['href']) for a in soup.find_all('a', href=re.compile(r"https://drive\.google\.com/.*"))]

    # Directory to save the file
    processed_dir = '../data/processed/'
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)

    # Save the links and titles to a file
    with open(os.path.join(processed_dir, 'google_drive_links.txt'), 'w') as file:
        for title, link in drive_links:
            file.write(f"{title}: {link}\n")
        print(f"Google Drive links saved to {os.path.join(processed_dir, 'google_drive_links.txt')}")

else:
    print("Failed to login.")


Logged in successfully.
Google Drive links saved to ../data/processed/google_drive_links.txt


In [ ]:
import gdown
import os
import re

# Function to create subdirectories based on grade and thematic area
def create_subdirectory(download_dir, filename):
    grade = re.search(r'GRADE \d+', filename, re.IGNORECASE)
    thematic_area = re.search(r'(SCHEMES|EXAMS|ASSIGNMENTS|NOTES|LESSON PLANS|CURRICULUM|ASSESSMENT AND SCORESHEET|TERM \d+)', filename, re.IGNORECASE)
    
    subdirectory = 'Miscellaneous'
    if grade and thematic_area:
        subdirectory = f"{grade.group(0).capitalize()}/{thematic_area.group(0).capitalize()}"
    
    subdirectory_path = os.path.join(download_dir, subdirectory)
    if not os.path.exists(subdirectory_path):
        os.makedirs(subdirectory_path)
    
    return subdirectory_path

# # Paths
# file_path = '/mnt/data/processed/cur_links.txt'  # Update with the correct file path
# download_directory = '/mnt/data/cur_downloads/'
# download_log_path = '/mnt/data/interim/cur_downloaded_files.log'
# error_log_path = '/mnt/data/interim/cur_error_files.log'


# Paths
file_path = '../data/processed/math_links.txt'
download_directory = '../data/math_downloads/'
download_log_path = '../data/interim/math_downloaded_files.log'
error_log_path = '../data/interim/math_error_files.log'

# Regular expression pattern for file and folder URLs
file_pattern = r'drive\.google\.com/file/d/([a-zA-Z0-9_-]+)/view'
folder_pattern = r'drive\.google\.com/drive/folders/([a-zA-Z0-9_-]+)'

# Ensure download directory exists
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Load downloaded files log
if os.path.exists(download_log_path):
    with open(download_log_path, 'r') as log_file:
        downloaded_files = log_file.read().splitlines()
else:
    downloaded_files = []

# Load error log
if os.path.exists(error_log_path):
    with open(error_log_path, 'r') as error_file:
        failed_downloads = error_file.read().splitlines()
else:
    failed_downloads = []

# Process links
with open(file_path, 'r') as file:
    for line in file:
        if line in downloaded_files or line in failed_downloads:
            continue  # Skip if already processed

        file_match = re.search(file_pattern, line)
        folder_match = re.search(folder_pattern, line)

        if file_match:
            try:
                file_id = file_match.group(1)
                filename = line.split(':')[0].strip()
                download_link = f'https://drive.google.com/uc?id={file_id}&export=download'
                
                subdirectory_path = create_subdirectory(download_directory, filename)
                output_path = os.path.join(subdirectory_path, filename)
                
                gdown.download(download_link, output_path, quiet=False, use_cookies=False)
                
                with open(download_log_path, 'a') as log_file:
                    log_file.write(f"{line}\n")
            except Exception as e:
                print(f"Failed to download {line.strip()}. Error: {e}")
                with open(error_log_path, 'a') as error_file:
                    error_file.write(f"{line}\n")
        elif folder_match:
            try:
                folder_id = folder_match.group(1)
                folder_url = f"https://drive.google.com/drive/folders/{folder_id}"
                
                folder_download_path = os.path.join(download_directory, "Folders")
                if not os.path.exists(folder_download_path):
                    os.makedirs(folder_download_path)
                
                gdown.download_folder(folder_url, output=folder_download_path, quiet=False, use_cookies=False)
                
                with open(download_log_path, 'a') as log_file:
                    log_file.write(f"{line}\n")
            except Exception as e:
                print(f"Failed to download folder {line.strip()}. Error: {e}")
                with open(error_log_path, 'a') as error_file:
                    error_file.write(f"{line}\n")
        else:
            print(f"Could not extract file ID from line: {line}")


In [ ]:
import gdown
import os
import re

# Function to create subdirectories based on grade and thematic area
def create_subdirectory(download_dir, filename):
    grade = re.search(r'GRADE \d+', filename, re.IGNORECASE)
    thematic_area = re.search(r'(SCHEMES|EXAMS|ASSIGNMENTS|NOTES|LESSON PLANS|CURRICULUM|ASSESSMENT AND SCORESHEET|TERM \d+)', filename, re.IGNORECASE)
    
    subdirectory = 'Miscellaneous'
    if grade and thematic_area:
        subdirectory = f"{grade.group(0).capitalize()}/{thematic_area.group(0).capitalize()}"
    
    subdirectory_path = os.path.join(download_dir, subdirectory)
    if not os.path.exists(subdirectory_path):
        os.makedirs(subdirectory_path)
    
    return subdirectory_path

# Paths
file_path = '../data/processed/cur_links.txt'
download_directory = '../data/cur_downloads/'
download_log_path = '../data/interim/cur_downloaded_files.log'
error_log_path = '../data/interim/cur_error_files.log'

# Regular expression pattern to extract file ID from Google Drive link
pattern = r'drive\.google\.com/file/d/([a-zA-Z0-9_-]+)/view'

# Ensure download directory exists
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Load downloaded files log
if os.path.exists(download_log_path):
    with open(download_log_path, 'r') as log_file:
        downloaded_files = log_file.read().splitlines()
else:
    downloaded_files = []

# Load error log
if os.path.exists(error_log_path):
    with open(error_log_path, 'r') as error_file:
        failed_downloads = error_file.read().splitlines()
else:
    failed_downloads = []

# Process links
with open(file_path, 'r') as file:
    for line in file:
        if line in downloaded_files or line in failed_downloads:
            continue  # Skip if already processed

        match = re.search(pattern, line)
        if match:
            try:
                file_id = match.group(1)
                filename = line.split(':')[0].strip()
                download_link = f'https://drive.google.com/uc?id={file_id}'
                
                subdirectory_path = create_subdirectory(download_directory, filename)
                output_path = os.path.join(subdirectory_path, filename)
                
                gdown.download(download_link, output_path, quiet=False)
                
                with open(download_log_path, 'a') as log_file:
                    log_file.write(f"{line}\n")
            except Exception as e:
                print(f"Failed to download {line.strip()}. Error: {e}")
                with open(error_log_path, 'a') as error_file:
                    error_file.write(f"{line}\n")
        else:
            print(f"Could not extract file ID from line: {line}")
